# tensorflow

In [9]:
import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets

## Preprocess

In [11]:
(train_x , train_y), (test_x, test_y) = datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [13]:
train_x = train_x[..., tf.newaxis]
test_x = test_x[..., tf.newaxis]

train_x = train_x / 255.
test_x = test_x / 255.

##  하이퍼파라미터 튜닝

In [24]:
input_shape = (28,28,1)
num_classes = 10

learning_rate = 0.001

dropout_rate = 0.7

num_epochs = 1
batch_size = 64


## Build Model

In [21]:
inputs = layers.Input( input_shape )
net = layers.Conv2D(32, (3,3), padding = 'SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3,3), padding = 'SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size = (2,2))(net)
net = layers.Dropout(0.5)(net)

net = layers.Conv2D(64, (3,3), padding = 'SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3,3), padding = 'SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size = (2,2))(net)
net = layers.Dropout(0.5)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(0.5)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs = inputs , outputs = net, name ="Basic_CNN")

In [25]:
#Model is the full model w/o custom layers
model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate),
                 loss = 'sparse_categorical_crossentropy',
                 metrics = ['accuracy'])

## Training 

In [26]:

model.fit(train_x, train_y, batch_size = batch_size , shuffle= True)

model.evaluate(test_x , test_y, batch_size= batch_size)

157/157 [==============================] - 3s 21ms/step - loss: 2.3011 - accuracy: 0.1135


[2.301063060760498, 0.11349999904632568]

# PyTorch

In [2]:
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets , transforms

In [3]:
seed = 1

lr = 0.001
momentum = 0.5

batch_size = 64
test_batch_size = 64

epochs = 1
no_cuda = False
log_interval = 100  # 100 번 돌때마다 log 를 보여주겠다 ! - tf 는 프로그래스바 제공해줌 torch 는 안줌.. 우리가 짜야함 

## Model

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,20,5,1)
        self.conv2 = nn.Conv2d(20, 50 ,5 ,1)
        self.fc1 = nn.Linear(4 *4 *50, 500)
        self.fc2 = nn.Linear(500, 19)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x  = F.max_pool2d(x , 2, 2)
        x = F.relu(self.conv2(x))
        x  = F.max_pool2d(x , 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x , dim = 1)

## Preprocess

In [5]:
torch.manual_seed(seed)

use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

kwargs = {'num_workers' : 1 , 'pin_memory' : True} if use_cuda else {}

train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('dataset', train = True , download = True ,
                        transform = transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.1307,),(0.3081,))
                        ])),
        batch_size = batch_size , shuffle= True , **kwargs)


test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('dataset', train = False  ,
                        transform = transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.1307,),(0.3081,))
                        ])),
        batch_size = batch_size , shuffle= True , **kwargs)

## Optimization  

In [6]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr = lr , momentum= momentum)

## Training 

In [7]:
for epoch in range(1, epochs +1):
    #Train model
    model.train()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device),  target.to(device)
        optimizer.zero_grad() # backpropagation 계산하기 전에 0으로 기울기 계산
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward() # 계산한 기울기
        optimizer.step()
        
        if batch_idx & log_interval == 0:
            print("Train Epoch : {} [{}/{} ({:.0f}%)] Loss :{:.6f}".format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
            
    #Test mode
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device) , target.to(device)
            output = model(data)
            test_loss += F.null_loss(output, target, reduction = 'sum' ).item() #sum up batch loss
            pred = output.argmax(dim = 1 , keepdim = True ) #get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item() # pred와 target 이 같은지 확인
            
    test_loss /= len(test_loader.dataset)

    
    print("Test set : Average loss : {:.4f}, Accuracy: {}/{} ({:.0f}%)".format(
            test_loss, correct , len(test_loader.dataset),
            100 * correct / len(test_loader.dataset)))

Train Epoch : 1 [0/60000 (0%)] Loss :2.929361
Train Epoch : 1 [64/60000 (0%)] Loss :2.929743
Train Epoch : 1 [128/60000 (0%)] Loss :2.941966
Train Epoch : 1 [192/60000 (0%)] Loss :2.914491
Train Epoch : 1 [512/60000 (1%)] Loss :2.898558
Train Epoch : 1 [576/60000 (1%)] Loss :2.891137
Train Epoch : 1 [640/60000 (1%)] Loss :2.902653
Train Epoch : 1 [704/60000 (1%)] Loss :2.896774
Train Epoch : 1 [1024/60000 (2%)] Loss :2.877575
Train Epoch : 1 [1088/60000 (2%)] Loss :2.882554
Train Epoch : 1 [1152/60000 (2%)] Loss :2.869534
Train Epoch : 1 [1216/60000 (2%)] Loss :2.866179
Train Epoch : 1 [1536/60000 (3%)] Loss :2.853128
Train Epoch : 1 [1600/60000 (3%)] Loss :2.831231
Train Epoch : 1 [1664/60000 (3%)] Loss :2.852648
Train Epoch : 1 [1728/60000 (3%)] Loss :2.848146
Train Epoch : 1 [8192/60000 (14%)] Loss :2.367702
Train Epoch : 1 [8256/60000 (14%)] Loss :2.348148
Train Epoch : 1 [8320/60000 (14%)] Loss :2.343766
Train Epoch : 1 [8384/60000 (14%)] Loss :2.329055
Train Epoch : 1 [8704/60000

AttributeError: module 'torch.nn.functional' has no attribute 'null_loss'